# ELEC3305 Lab Project - Time Domain Sonar Lab


In this lab, we will interact with physical time-domain signals. We will use the chirp signal to characterize the response of the speaker-microphone system and look at detecting signals using cross-correlation.
In the second part, we will build on part one and use the speaker-microphone system to develop a simple sonar.


In [1]:
# Import functions and libraries
import numpy as np, matplotlib.pyplot as plt
import threading,time, queue, pyaudio 
from matplotlib.pyplot import *
import matplotlib.cm as cm
from scipy import signal
from threading import Lock


%matplotlib inline

## Part 1: Chirping!

When playing a sound and recording, the signal goes through several systems. In particular it goes through the response of the soundcard output, the speaker, the room we are in and the response of the microphone and receive part of the USB soundcard.

A chirp is a a signal in which the frequency increases linearly with time. In this assignment we will generate a chirp signal and use it to measure the amplitude of the frequency response of our speaker-room-microphone system. This lab will work best in a quiet environment -- We recommend that you execute the lab at home or in a quiet place before submitting it. 

A simultaneous frequency is defined as the derivative of the phase of a signal, $f = \frac{1}{2\pi} \frac{d\phi (t)}{ dt} $. For example, the simultaneous frequency of $\cos(\phi(t))=\cos(2\pi f_0 t)$ is  

$$f = \frac{d\phi (t)}{2\pi dt}  = f_0$$ 


For a linear chirp, the frequency changes linearly over time. The simultaneous frequency is therefore defined as 

$$ f(t) = f_0 + kt. $$


So,  

$$ x(t) = \sin(2\pi\int_0^t f(t')dt') = \sin(2\pi\int_o^t(f_0+kt')dt') = \sin(2\pi(f_0+\frac{k}{2}t)t) $$


### Part I Task I: Generating the Chirp

Generate a 10 seconds long chirp signal, sampled at 48,000[Hz] with a frequency range of 20[Hz] to 20,000[Hz]. Set the magnitude of the chirp to 0.5. This will help prevent non-linearities when we play the sound later. 

* Given $T$=total time length, $f_0$=start frequency, $f_1$ = end frequency, derive a formula $f(t)$ for the frequency sweep. Write the formula here:

$$m = (f_1 - f_0)/T \\ f(t) = f_0 + mt \\ f(t) = f_0 + \frac{f_1 - f_0}{T} t \:\text{Hz}$$

* Find the formula for the phase by integrating $\phi(t) = \int_0^T f(t)dt$ to get the phase function. Write the formula here:

$$
\frac{\phi(t)}{2\pi} = \int_0^T f(t)dt \\ = \int_0^T f_0 + \frac{f_1 - f_0}{T} t dt \\ = f_0t + \frac{f_1 - f_0}{2T} t^2 
$$

Now, 
* Set the sample-rate frequency `fs = 48000` Hz
* Generate a time index from `t=0` to `t=10` with sampling rate of 48000 Hz
* Generate a vector of frequency vs time: `phi_of_t`  (  $\phi(t)$ )
* Generate the chirp function `s_chirp` with amplitude of 0.5 by plugging the phase into a sinusoid. 
  
  


In [2]:
fs = 48000
f0 = 20
f1 = 20000
#f2 = 20
T = 10
# your code here

A = 0.5

# generate time index and phase
t_values = np.r_[0.0:T*fs:1.0]/fs

m = (f1 - f0) / T
ft = np.vectorize(lambda t : m*t + f0)
phi_t = np.vectorize(lambda t : m / 2 * t**2 + f0 * t)

# generate chirp signal
x_t = np.vectorize(lambda t : A*np.sin(2*np.pi * phi_t(t)))
chirp_signal = x_t(t_values)


* Plot the first $\frac{1}{2}$ second of the chirp (`s_chirp`), you will notice that the carrier frequency increases and that the chirp has a constant envelope. To get a nice figure, make sure the aspect ratio of the figure is height/width = 0.2 . Label the axis and figure appropriately. 

In [ ]:
# Set the aspect ratio such that the image is wide
width, height = figaspect(0.2)
fig = figure(figsize=(width,height))

#Your code below:
half_sec_n_samples = int(0.5*fs)
plt.plot(t_values[:half_sec_n_samples], chirp_signal[:half_sec_n_samples])
plt.xlabel('Time')
plt.ylabel('Chirp')
plt.show()

* Plot the magnitude frequency response of the sequence from 0 to $\pi$ using the function `signal.freqz`. Note, that the digital frequency range represents a physical frequency range of 0[hz] to 24000[Hz]. To get a nice figure, make sure the aspect ratio of the figure is height/width = 0.2. Label the axis and figure appropriately. 

The `signal.freqz` function can be slow -- be patient. 

In [ ]:
# # generate frequency response of chirp
N = 1000
w, h = signal.freqz(chirp_signal, worN=N)

f_values = np.linspace(0,np.pi,N)
f_gap = np.pi/N
stop_index = int(fs/2 / f_gap)


# generate frequency index
f_values = w*fs/(2*np.pi)
mag = np.absolute(h)

# plot
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.plot(f_values[:stop_index], mag[:stop_index])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Chirp Frequency Response Magnitude')
plt.show()



Explain why the chirp is an appropriate signal to measure the magnitude frequency response of a system. 


#### Your answer here:

We want to accurately measure the frquency response amplitude of the microphone speaker setup, which can be done using our chirp signal. This chirp allows us to see how the system can play a sound through a computer's sound card and speakers into the room before the response is recorded by the microphone and received by the sound card.

The chirp has an approximately uniform magnitude over a wide range of frequencies in that it represents a rect function in the frequency domain. Therefore, we can imagine the transfer function for the system (speaker-room-microphone) simply multiplying this rect function, which is far easier to visualise and mathenmatically calculate this transfer function.

### Part I Task II: Playing and Recording the Chirp
Now, we will play the sound of the chirp on our computer speaker and simultaneously record using the microphone. 

* On Apple computers it is recommended that you turn off the ambient noise reduction by going to system-preferences, selecting sound, choose the input tab and make sure that the "Use ambient noise reduction" box is unchecked. In some windows system there's ambient noise reduction as well. Make sure it is also turned off. 

* Your laptop most likely has two speakers. It is best if we work only with one. Go to the operating system's sound settings and change the stereo settings such that the speaker that is closest to the microphone is active. Your result will be much better that way. 
		
* Make sure your output volume is at 70-80% and that the laptop's microphone is on, again to avoid non-linear distorsions. 

* We will record 12 seconds just to make sure we capture the entire sequence. 

The code below defines some functions to use with pyaudio -- a multi-platform audio python interface. 

In [3]:
def play_audio( Q, p, fs , dev=None):
    # play_audio plays audio with sampling rate = fs
    # Q - A queue object from which to play
    # p   - pyAudio object
    # fs  - sampling rate
    # dev - device number
    
    # Example:
    # fs = 44100
    # p = pyaudio.PyAudio() #instantiate PyAudio
    # Q = Queue.queue()
    # Q.put(data)
    # Q.put("EOT") # when function gets EOT it will quit
    # play_audio( Q, p, fs,1 ) # play audio
    # p.terminate() # terminate pyAudio
    
    # open output stream
    ostream = p.open(format=pyaudio.paFloat32, channels=1, rate=int(fs),output=True,output_device_index=dev)
    # play audio
    while (1):
        data = Q.get()
        if (data) == 'EOT' :
            break
        try:
            ostream.write( data.astype(np.float32).tobytes() )
        except:
            break
            
def record_audio( queue, p, fs ,dev=None,chunk=2048,lock=None):
    # record_audio records audio with sampling rate = fs
    # queue - output data queue
    # p     - pyAudio object
    # fs    - sampling rate
    # dev   - device number
    # chunk - chunks of samples at a time default 1024
    #
    # Example:
    # fs = 44100
    # Q = Queue.queue()
    # p = pyaudio.PyAudio() #instantiate PyAudio
    # record_audio( Q, p, fs, 1) #
    # p.terminate() # terminate pyAudio
    
    istream = p.open(format=pyaudio.paFloat32, channels=1, rate=int(fs),input=True,input_device_index=dev,frames_per_buffer=chunk)

    # record audio in chunks and append to frames
    frames = [];
    while (1):
        try:  # when the pyaudio object is destroyed, stops
            with lock if lock is not None else 1:
                data_str = istream.read(chunk, exception_on_overflow=False) # read a chunk of data
        except:
            break
        data_flt = np.frombuffer( data_str, 'float32' ) # convert string to float
        queue.put( data_flt ) # append to list

def xciever(sig, fs):
    # function takes a signal and a sampling frequency
    # it then plays and records at the same time. The function returns
    # the recorded sound.

    rcv = [];

    # create an input output FIFO queues
    Qin = queue.Queue()
    Qout = queue.Queue()

    #lock for controlling access to shared resources
    lock = Lock()
    
    # create a pyaudio object
    p = pyaudio.PyAudio()

    # initialize a recording thread.
    t_rec = threading.Thread(target = record_audio,   args = (Qin, p, fs ), kwargs={'lock': lock})
    t_play_audio = threading.Thread(target = play_audio,  args = (Qout, p, fs  ))

    # start the recording and playing threads
    t_rec.start()
    t_play_audio.start()

    Qout.put( sig );
    Qout.put( "EOT" );

    # pause for RECORD_SECS seconds
    RECORD_SECS = len(sig)/fs + 2.0
    time.sleep( RECORD_SECS )
    # terminate pyAudio
    with lock:
        p.terminate()
        
    # append to output
    while ( not Qin.empty()) :
        data = Qin.get()
        rcv = np.append( rcv, data )

    return rcv

__Playing and recording audio:__

* Run the following code. It is an example of how to play and record sound at the same time and uses threading for the play and record threads.

The resulting received sequence will be stored in the variable `rcv_chirp`.

In [6]:
## Play and record chirp at the same time

fs = 48000 # sampling rate = 48000 Hz

rcv_chirp = xciever( chirp_signal, fs) # Qin = queue.Queue() Qout = queue.Queue() / Note: queue instead of Queue for python3


C:\Users\lewis_000\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:22: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


* Plot the frequency response of the received sequence. 
* Also, plot the absolute value of the received signal. Plotting the absolute value (sort of) displays the envelope of the chirp. 

Label the figures and use an aspect ratio of Height/Width = 0.2


In [ ]:
## Plot chirp response


# generate frequency response of recorded chirp
w, h = signal.freqz(rcv_chirp, 1, 512)

# generate frequency index
freq = fs*w/(2*np.pi)

# generate a time index
t = np.r_[0:(len(rcv_chirp)/fs):(1/fs)]


# free code for your plot:
width, height = figaspect(0.2)
fig1 = figure(figsize=(width,height))
plt.plot(freq, abs( h ) )
plt.title('Frequency response of the transceived chirp (Hz)')
plt.xlabel('f[Hz]')

fig1 = figure(figsize=(width,height))
plt.plot(t[0:len(rcv_chirp)], abs(rcv_chirp))
plt.title('Absolute value of transceived chirp');
plt.xlabel('time[s]')

* Comment on the results you got. In addition, what is the implicit assumption we are making in order to claim that the result is a frequency response? 
(HINT: consider the case when the chirp was very short)

#### Answers here:

The time response visually resembles the frequency response. This makes intuitive sense, as the frequency is made a linear function of time by the phase function. There the envelope of the chirp consists of various peaks and troughs up to 10 kHz. Beyond this, the signal reduces close to the noise floor due to the operating frequency ranges of the computer's speakers and microphones. Note that as it depends on hardware *this range would vary per system*. Below 10 kHz, certain frequencies appear to have been absorbed or reflected by the room, with absorbtion (or at least, the lack of reflection of room acoustics) characterised by the troughs. Note that the system's microphone and speaker would also contribute frequency responses in this system, however, these are unlikely to cause the troughs.

The implicit assumption we are making in order to claim that the result is a frequency response is that the frequency response of the room does not change over time. For the long 10 second chirp, if the response changes during this time, the resulting measured frequency response will be a mix of different responses over the different frequency ranges (not averaged!) Thus we can only measure the response accurately when the response does not change over the 10 seconds - or in the sonar's case, where objects are not moving too fast.

### Part I, Task III: Envelope detection with Hilbert transform. 
The absolute value of the of the result "sort of" displays the envelope, however it is still modulated by the (now rectified) frequency sweep carrier. If we write down the response, it can be expressed approximately as 

$$y[n] = |H[n]| \sin(2\pi (f_0 +k[n*T])nT + \angle H[n])$$

where $|H[n]|$ is the frequency response for the instantaneous frequency at the nth sample and $\angle H[n]$ is its phase response. 

The reason that it is only an approximation is that there is an inherent assumption that we do not look at transient effects, only steady state effect for each frequency. This is a good approximation because our chirp is very slow compared to the propagation of sound in the room. 

One way to get the envelope $|H[n]|$ is to convert it to its analytic signal. The analytic signal $x_a(t)$ of signal $x(t)$ is:

$$x_a = F^{-1}(F(x)\cdot 2U) = x + j y$$

where $F$ is the Fourier transform, $U$ the unit step function,
and $y$ "is" the Hilbert transform of $x$. In other words, the negative half of the frequency spectrum is zeroed
out, turning the real-valued signal into a complex signal.

The analytic signal of the received chirp will then be: 

$$ y_a[n] = |H[n]|e^{j2\pi (f_0 +k[n*T])nT + \angle H[n]} $$

The envelope can be detected by taking the magnitude. 
(analytic function of y seems to have a one more phase shift $\pi/2$ since it is a $\sin$ function)

* Compute the analytic signal by using the function `signal.hilbert` and plot its absolute value. Note that the discrete hilbert transform is not perfect, since it uses FIR filtering. This will show up as ripple in the envelope.

* Label the figures and use an aspect ration of Height/Width = 0.2

In [ ]:
width, height = figaspect(0.2)
fig1 = figure(figsize=(width,height))

## Your lovely code here:
rcv_chirp_a = rcv_chirp + 1j*signal.hilbert(rcv_chirp)
env = abs(rcv_chirp_a)

#Create time scale
t = np.r_[0:(len(env)/48000):(1/48000)]

#plot:
plt.plot(t[0:len(env)],  env  )
plt.title('Absolute envelope of transceived chirp $|H[n]|$')
plt.xlabel('time[s]')

### Part I, Task IV: Auto-correlation Properties of the Chirp:

In part II of the lab, we will be sending and receiving chirp pulses to estimate delays between the tranceived pulses. This is done by cross correlating / matched filtering the received signal with the known chirp pulse to detect the echoes. In this task, we will investigate the correlation properties of the chirp.

A cross correlation is defined as:

$$ R_{xy}[n] = \sum_{m=-\infty}^\infty x[m]y^*[m-n] = (x[m]*y^*[-m])[n]$$

, where $y^*[-m]$ is the complex conjugat of $y[-m]$. This similar to a convolution, without flipping one of the signals. It can be implemented using a convolution as shown above. In general, the more correlated the two signals is at position $n$, the higher the value will be. That's why it is useful in a sonar system.

#### Matched filter 
When we look for a very specific shape in a signal, we can compute a cross correlation between the signal and the shape we are interested in. In that case, the operation of the cross correlation is also called a matched filter -- i.e. correlating with a filter that is matched to the shape we look for. 

Because we will be doing cross-correlations between a chirp pulse and its echoes, it is useful to look at the auto-correlation, which is basically a cross correlation of the signal with itself. A discrete autocorrelation of a signal is defined as: 

$$ R_{xx}[n] = \sum_{m=-\infty}^\infty x[m]x^*[m-n] = (x[m]*x^*[-m])[n]$$ 

The chirp has a very nice property that its auto-correlation is very narrow. Since the spread of the resulting correlation determines how fast you can detect, the width of the auto-correlation is important. This property is called pulse compression and is widely considered in radar design. Random noise and some other pseudo-random like sequences also possess this property. 

* Generate a 512 sample chirp pulse with a frequency sweep from 17KHz-18KHz and sampling rate fs=48000. 
* Validate its frequency response by plotting it.

In [ ]:
## Your beautiful code here:
# Sampling rate and points
n = 512
fs = 48000
T = n/fs
# Frequency sweep variables 
f_initial = 17000
f_final = 18000

# generate time axis
tAxis = np.r_[0:n/fs:1/fs]

# generate chip signal
f_of_t = (f_final - f_initial)/(2*T)*tAxis + f_initial # m*t + f0
phi_of_t = 2*np.pi*f_of_t*tAxis # m*t^2 + f0
s_chirp = np.sin(phi_of_t)

# generate frequency response of chirp
w, chirpfft = signal.freqz(s_chirp, worN=n)
freqResponse = np.r_[0:n]/(2*T)


# plot the signal
plt.figure()
width, height = figaspect(0.2)
fig = figure(figsize=(width,height))
plt.plot(tAxis[0:len(s_chirp)]*1000, s_chirp)
plt.xlabel( "Time (ms)" )
plt.ylabel( "Amplitude" )
plt.title("512 sample chirp pulse with a frequency sweep from 17KHz-18KHz")


# Plot the FFT of the signal
plt.figure()
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
#plt.plot(freqResponse,chirpfft.real)
#plt.plot(freqResponse,chirpfft.imag)
plt.plot(freqResponse[0:len(chirpfft)],abs(chirpfft))
#plt.legend(["Real","Imaginary","Magnitude"])
plt.xlabel("frequency (kHz)")
plt.ylabel("Amplitude")
plt.title("FFT of 512 sample chirp pulse")
plt.minorticks_on()
axvline(x=f_initial, color='r', linestyle='--')
axvline(x=f_final, color='r', linestyle='--')


* Compute the autocorrelation of the chirp "using" discrete convolution, either with `signal.convolve` or `signal.fftconvolve`. Remember that you have to flip the signal since convolution does that already. You can flip a signal `x` by doing `x[::-1]`. Use mode=''full'' for convolution.
* Plot the autocorrelation. Your plot should be spiky because we did not do envolope detection yet. Use miliseconds as the x-axis

In [ ]:
## Your fantastic code here:

# Flip the signal as instructed
s_chirp_flipped = s_chirp[::-1]

# Compute the autocorrelate of chirp using discrete convolution
s_chirp_autocorrelate = signal.convolve(s_chirp, s_chirp_flipped, mode='full', method='auto')
# Determine size of s_chirp_autocorrelate
size = s_chirp_autocorrelate.size
# Create new t axis for autocorrelation
tAxisAutocorrelate = np.r_[0:size] * 1000 / fs


# Plot the autocorrelation
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.plot(tAxisAutocorrelate, s_chirp_autocorrelate)
plt.xlabel('Time (ms)')
plt.ylabel('Value')
plt.title('Autocorrelation $R_{xx}[n]$ of 512-Sample Chirp Pulse (17 kHz to 18 kHz, fs= 48 kHz)')
plt.show()
fig = plt.figure(figsize=(width,height))
plt.plot(tAxisAutocorrelate, np.abs(s_chirp_autocorrelate))
plt.xlabel('Time (ms)')
plt.ylabel('Magnitude')
plt.title('Autocorrelation $|R_{xx}[n]|$ of 512-Sample Chirp Pulse (17 kHz to 18 kHz, fs= 48 kHz)')
plt.show()

In a similar way as we did before, it is possible to recover the envelope of the autocorrelation by performing a cross-correlation with the analytic signal and then taking the absolute value. In this case, we know exactly what is the analytic function is!

* Generate `s_chirp_a`, the analytic function of the chirp by computing: `s_chirp_a = exp(1j* phi_of_t )`. Perform cross correlation between `s_chirp_a` and `s_chirp` and show the envelope. As stated before, this could also be called a matched filter. 
* Measure the full-width at half max (FWHM) of the main lobe of the autocorrelation. 
* Comment on the FWHM of the main lobe of the matched-filter with respect to the length of the pulse. That ratio is also called pulse compression.  For simplicity, normalize the plot such that the maximum is 1, but record the maximum value of the autocorrelation and display it in the title of the figure.  

Use the pragma ``%matplotlib notebook`` for making the figure interactive, and ``plt.grid('on')`` for displaying a grid. 

Use miliseconds as the x-axis



In [ ]:
# your nice script to produce beautiful chirps, xcorralations and figures here:
s_chirp_a = np.exp(1j*phi_of_t)
xcorr = signal.fftconvolve(s_chirp, s_chirp_a[::-1], mode='full')
env = abs(xcorr)

# max value
max_value = np.amax(env)
print('Max Value = ' + str(max_value))

# fwhm
peak_i = np.argmax(env)
half_value = max_value/2
top_half_i = np.where(env >= half_value)
fwhm = (top_half_i[0][-1] - top_half_i[0][0])/fs *1000
print('FWHM = %.4f ms'%fwhm)
print('Chirp pulse length = %.4f ms'%(T*1000))

# normalise
env = env/max_value

# time
t = np.r_[0:len(env)/fs:(1/fs)]*1000

# plot
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width, height))
plt.plot(t[0:len(env)], env)
plt.title('Envelope of the Normalised Autocorrelation of the 512-Sample Chirp Pulse. Unnormalised Maximum Value = 255.8421')
plt.xlabel('Time (ms)')
plt.grid('on')
plt.minorticks_on()
axhline(y=0.5, color='r', linestyle='--')

**Your answer here:**

With the FWHM of 1.2083 ms and the chirp pulse length of 10.6667 ms, the FWHM of the main lobe is 11.33% of the length of the pulse, giving a pulse compression of $\frac{10.6667\:\text{ms}}{1.2083\:\text{ms}}$ = 8.827. Thus a fair compression is achieved.
_____

Now we will look at why the chirp pulse is better for cross-correlation detection than a pure tone.
- Repeat Task "III" <- "IV" for:
 1. A constant frequency of 17000Hz, 512 samples in length. 
 2. A chirp with a frequency sweep from 16500Hz - 17500Hz (1KHz Bandwidth), 512 in length.  
 3. A chirp with a frequency sweep from 15000Hz - 19000Hz (4KHz Bandwidth), 512 in length
- Make three subplots in one figure 
- Compare the size of the main lobes (full width at half max). How much "Pulse Compression" are you getting by using a chirps for detection compared to a single frequency pulse?
- What is the approximate bandwidth of the pure frequency pulse and what is the bandwidth of the chirp pulses? Comment on the tradeoff between bandwidth and pulse compression
- What is the maximum autocorrelation for each pulse?


   

In [ ]:
# your solution here
fs = 48000
T = 512/fs

t = np.r_[0:T:(1/fs)]

f_0 = 17000
phi_pure = 2*np.pi*(f_0)*t
pure_chirp = np.sin(phi_pure)
pure_chirp_a = np.exp(1j*phi_pure)
pure_xcorr = abs(signal.fftconvolve(pure_chirp, pure_chirp_a[::-1], mode = 'full'))

# chirp 1
m1 = (17500-16500)/T
t_chirp = np.r_[0:T:(1/fs)]
phi_t1 = 2*np.pi*(16500+m1*t/2)*t
chirp_1 = np.sin(phi_t1)
chirp_1a = np.exp(1j*phi_t1)

xcorr_1 = abs(signal.fftconvolve(chirp_1, chirp_1a[::-1], mode = 'full'))

# chirp 2
m2 = (19000-15000)/T
phi_t2 = 2*np.pi*(15000+m2*t/2)*t
chirp_2 = np.sin(phi_t2)
chirp_2a = np.exp(1j*phi_t2)

xcorr_2 = abs(signal.fftconvolve(chirp_2, chirp_2a[::-1],mode='full'))

# Double time for correlation results:
t = np.r_[0:2*T:(1/fs)]

# plot
width, height = figaspect(0.3)
fig1, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(width,height))
plt.tight_layout()
plt.subplots_adjust(top=1.5)
ax_data = ((ax1,'Pure Tone 17000Hz',pure_xcorr), (ax2,'Chirped Tone 16500Hz - 17500Hz (1KHz Bandwidth)',xcorr_1),(ax3,'Chirped Tone 15000Hz - 19000Hz (4KHz Bandwidth)',xcorr_2))
for ax_d in ax_data:
    axis, title, env = ax_d
    # fwhm
    peak_i = np.argmax(env)
    max_value = np.amax(env)
    half_value = max_value/2
    top_half_i = np.where(env >= half_value)
    fwhm = (top_half_i[0][-1] - top_half_i[0][0])/fs *1000
    print('FWHM of %s = %.4f ms'%(title,fwhm))
    print('Max Autocorrelation Value of %s = %.4f'%(title,max_value))

    # normalise
    env /= max_value

    # plot
    axis.plot(t[0:len(env)]*1000, env)
    axis.set_title('Aurocorrelation of %s'%title)
    axis.set_xlabel('Time (ms)')
    axis.grid('on')
    axis.axhline(y=0.5, color='r', linestyle='--')

**Your answers and interpretations here:**

FHWM results:
* The pulse length of 512 samples at 48 kHz is 10.6667 ms
* FWHM of Pure Tone = 10.6458 ms
  * Pulse Compression is $\frac{10.6667\:\text{ms}}{10.6458\:\text{ms}} = 1.002$
* FWHM of Chirped Tone 16500Hz - 17500Hz (1KHz Bandwidth) = 1.2083 ms
  * Pulse Compression is $\frac{10.6667\:\text{ms}}{1.2083\:\text{ms}} = 8.828$
* FWHM of Chirped Tone 15000Hz - 19000Hz (4KHz Bandwidth) = 0.2917 ms
  * Pulse Compression is $\frac{10.6667\:\text{ms}}{0.2917\:\text{ms}} = 36.567$

Clearly the size of the main lobes (full width at half max) decreases as the range of frequency increases. A single frequency pulse provides negligible compression and is therefore ineffective, in contrast to the 4 kHz chirp, which provided 36.6 times compression.

The pure sine pulse in the frequency domain approximately approaches a finite delta peak at 17 kHz and therefore has negligible bandwidth, compared to the pulses that have bandwidths of 1 kHz and 4 kHz respectively. Higher pulse compression requires more bandwidth to be sacrificed, thus marking the tradeoff.

The maximum value of each pulse's autocorrelation is very similar. There is a small increase in maxima with decreased bandwidth, with the pure tone having the highest maxima:
* Max Autocorrelation Value of Pure Tone 17000Hz = 255.4589
* Max Autocorrelation Value of Chirped Tone 16500Hz - 17500Hz (1KHz Bandwidth) = 255.4352
* Max Autocorrelation Value of Chirped Tone 15000Hz - 19000Hz (4KHz Bandwidth) = 255.3431

Notably, the pure tone has higher autocorrelation overall for the total time length, judging by the area under the autocorrelation function (integration). This is obvious as a pure sinusoidal is periodic and thus more similar to itself over time in general.

- Now, repeat task "III" <- "IV" for 
 1. a chirp with a frequency sweep from 15000Hz - 19000Hz, 256 in length
 2. a chirp with a frequency sweep from 15000Hz - 19000Hz, 512 in length

- Make two subplots in one figure 
- Compare the size of the main lobe (full width at half max) to the previous case of  15000Hz - 19000Hz, 512 in length.
- Compare the maximum autocorrelation as well. 

What's the effect of having more bandwidth? what's the effect of having longer/shorter pulses?



In [ ]:
# your solution here
fs = 48000
T1 = 256/fs
T2 = 512/fs

time1 = np.r_[0:T1:(1/fs)]
time2 = np.r_[0:T2:(1/fs)]

# chirp 1
m_t1 = (19000-15000)/T1
phi_t1 = 2*np.pi*(15000+m_t1*time1/2)*time1
chirp_t1 = np.sin(phi_t1)
chirp_t1_a = np.exp(1j*phi_t1)

xcorr_t1 = abs(signal.fftconvolve(chirp_t1, chirp_t1_a[::-1],mode='full'))

# chirp 2
m_t2 = (19000-15000)/T2
phi_t2 = 2*np.pi*(15000+m_t2*time2/2)*time2
chirp_t2 = np.sin(phi_t2)
chirp_t2_a = np.exp(1j*phi_t2)

xcorr_t2 = abs(signal.fftconvolve(chirp_t2, chirp_t2_a[::-1],mode='full'))

# Double times for correlation results:
time1 = np.r_[0:2*T1:(1/fs)]
time2 = np.r_[0:2*T2:(1/fs)]

# plot
width, height = figaspect(0.3)
fig1, (ax1, ax2) = plt.subplots(2, 1, figsize=(width,height))
plt.tight_layout()
plt.subplots_adjust(top=1.5)
ax_data = ((ax1,'Chirped Tone 256 samples (4 kHz Bandwidth)',xcorr_t1,time1), (ax2,'Chirped Tone 512 samples (4 kHz Bandwidth)',xcorr_t2,time2))
for ax_d in ax_data:
    axis, title, env, t = ax_d
    # fwhm
    peak_i = np.argmax(env)
    max_value = np.amax(env)
    half_value = max_value/2
    top_half_i = np.where(env >= half_value)
    fwhm = (top_half_i[0][-1] - top_half_i[0][0])/fs *1000
    print('FWHM of %s = %.4f ms'%(title,fwhm))
    print('Max Autocorrelation Value of %s = %.4f'%(title,max_value))

    # normalise
    env /= max_value

    # plot
    axis.plot(t[0:len(env)]*1000, env)
    axis.set_title('Aurocorrelation of %s'%title)
    axis.set_xlabel('Time (ms)')
    axis.grid('on')
    axis.axhline(y=0.5, color='r', linestyle='--')

####  Your answer below:

FWHM results:
* FWHM of Chirped Tone 256 samples (4 kHz Bandwidth) = 0.2917 ms
* FWHM of Chirped Tone 512 samples (4 kHz Bandwidth) = 0.2917 ms

Max Autocorrelation Value results:
* Max Autocorrelation Value of Chirped Tone 256 samples (4 kHz Bandwidth) = 127.9086
* Max Autocorrelation Value of Chirped Tone 512 samples (4 kHz Bandwidth) = 255.3431

The FWHM size of the main lobe for both pulses of size 256 and 512 samples is equal. Thus, halfing the length of the pulse has no effect on the compression ratio. The maximum autocorrelation differed dramatically compared to the previous section, with the 256-sample pulse resulting in half as much autocorrelation as the 512-sample pulse.

#### Summary
In summary, having more bandwidth decreases the FWHM of the main lobe and hence increases the pulse compression, while having longer/shorter pulses results in increased/decreased autocorrelation magnitude at these peaks.

### Dealing with sidelobes
As you can see, the chirp provides good pulse compression of the main-lobe. However, there exists very strong sidelobes. This is because the chirp is multiplied with a rect function, that is abrupt. Instead, we will window the chirp with one of the smooth window functions to taper off the sidelobes. 
    

* Repeat the above for a chirp with a sweep from 16.5KHz to 17.5KHz, and from 15KHz to 19KHz. This time, multiply the chirp (and its analytic function) with a hanning window.  You will find the function `np.hanning` useful. 

* plot the normalized autocorrelations (in the same figure) 
* Comment on the magnitude of the side-lobes? 
* Comment on the width of the main lobes? 
* What's the tradeoff?


In [ ]:
# your solution here
fs = 48000
T = 512/fs

t = np.r_[0:T:(1/fs)]

# chirp 1
m1 = (17500-16500)/T
t_chirp = np.r_[0:T:(1/fs)]
phi_t1 = 2*np.pi*(16500+m1*t/2)*t
chirp_1 = np.sin(phi_t1) * np.hanning(T*fs)
chirp_1a = np.exp(1j*phi_t1) * np.hanning(T*fs)

xcorr_1 = abs(signal.fftconvolve(chirp_1, chirp_1a[::-1], mode = 'full'))

# chirp 2
m2 = (19000-15000)/T
phi_t2 = 2*np.pi*(15000+m2*t/2)*t
chirp_2 = np.sin(phi_t2) * np.hanning(T*fs)
chirp_2a = np.exp(1j*phi_t2) * np.hanning(T*fs)

xcorr_2 = abs(signal.fftconvolve(chirp_2, chirp_2a[::-1],mode='full'))

# Double time for correlation results:
t = np.r_[0:2*T:(1/fs)]

# plot
width, height = figaspect(0.3)
fig1, (ax1, ax2) = plt.subplots(2, 1, figsize=(width,height))
plt.tight_layout()
plt.subplots_adjust(top=1.5)
ax_data = ((ax1,'Chirped Tone 16500Hz - 17500Hz (1KHz Bandwidth)',xcorr_1),(ax2,'Chirped Tone 15000Hz - 19000Hz (4KHz Bandwidth)',xcorr_2))
for ax_d in ax_data:
    axis, title, env = ax_d
    # fwhm
    peak_i = np.argmax(env)
    max_value = np.amax(env)
    half_value = max_value/2
    top_half_i = np.where(env >= half_value)
    fwhm = (top_half_i[0][-1] - top_half_i[0][0])/fs *1000
    print('FWHM of %s = %.4f ms'%(title,fwhm))
    print('Max Autocorrelation Value of %s = %.4f'%(title,max_value))

    # normalise
    env /= max_value

    # plot
    axis.plot(t[0:len(env)]*1000, env)
    axis.set_title('Aurocorrelation of %s With Hanning Window'%title)
    axis.set_xlabel('Time (ms)')
    axis.grid('on')
    axis.axhline(y=0.5, color='r', linestyle='--')

#### Your answers here:

Results:
* FWHM of Chirped Tone 16500Hz - 17500Hz (1KHz Bandwidth) = 2.4583 ms
* FWHM of Chirped Tone 15000Hz - 19000Hz (4KHz Bandwidth) = 0.6250 ms
* Max Autocorrelation Value of Chirped Tone 16500Hz - 17500Hz (1KHz Bandwidth) = 95.8125
* Max Autocorrelation Value of Chirped Tone 15000Hz - 19000Hz (4KHz Bandwidth) = 95.8125

The width of the main lobe is doubled:
* 2.4583 ms vs. 1.2083 ms
* 0.6250 ms vs. 0.2917 ms

The maximum autocorrelation is reduced:
* 95.8125 vs. 255.4352
* 95.8125 vs. 255.3431

But, the sidelobes are significantly smaller! Thus we tradeoff both width and maximum autocorrelation for no sidelobes.

### You are now ready to proceed to the Sonar Lab

## Part 2: Sonar

In this part of the lab we will write a simple application that implements a sonar using the laptop internal speaker and microphone. 

The basic idea is very simple and is the basis of sonar and ultrasound images -- Objects reflect sound waves. If we send a pulse of sound, we will get reflection echoes of that pulse. Detecting the echos and their time-of-flight will reveal their distance from the source, based on the speed of sound in air. 

The way we are going to implement the sonar is to generate a series of rapid pulses, and use matched filtering to detect the source and the returning echos. There are many parameters in this lab that can be tweaked to get different results. We encourage you to experiment. We hope you enjoy this. 


#### Instructions for laptops:

Unfortunately, the quallity of the sonar system is going to be highly dependent on your laptop quallity, and the position of the speakers and microphone. It is recommended that you adjust the sound settings on your system such that only the speaker that is closer to the microphone is active. For example, MacBookAirs have the microphone on the side of the computer -- so you should set adjust the audio settings to left speaker only. Also, it is recommended that the speaker volume be set to half of its maximum to avoid non-linear distortions.  

If you are getting poor results, please consult with us. 






### Part II, Task I: Generating Chirp Pulses

Recall from Part I, that the with of the main lobe of the autocorrelation depends on the bandwidth of the pulse. 
For a constant frequency pulse, the bandwidth will be inversly proportional to its length. Short pulses are localized in time, and therefore we will be able to separate echoes from targets that are close. However, short pulses carry less energy (for the same amplitude) and this will reduce our signal to noise ratio (SNR) in the detection and reduce our ability to detect the targets at all. So, in summary: for a constant frequency pulse, there's an inherent tradeoff between the resolution of the sonar (distinguish between close targets) and the signal to noise ratio. 

If we use a chirp pulse, we can increase the length of the pulse while also increasing the bandwidth. This will enable us to improve our signal to noise ratio as well as keeping the resolution of our sonar (by preserving the BW).

In our implemetation we are going to design a pulsed sonar system in which we repeatedly send pulses and then listen to the returning echoes. The arrival time of the echos will correspond to double the time-of-flight of sound propagation from our system to the target. 

* Write a function that generates a chirp pulse:
`pulse = genChirpPulse(Npulse, f0, f1, fs)` 

The function will accept: `Npulse` = number of samples, `f0,f1` = starting and ending frequency and `fs` = sampling frequency. The function will return the analytic function of the chirp $\exp (j 2\pi \int_0^t f(t)dt )$ with amplitude 1.

In [6]:
def genChirpPulse(Npulse, f0, f1, fs):
    #     Function generates an analytic function of a chirp pulse
    #     Inputs:
    #             Npulse - pulse length in samples
    #             f0     - starting frequency of chirp
    #             f1     - end frequency of chirp
    #             fs     - sampling frequency
    #     Output:
    #              pulse - chirp pulse
    
    # Create time axis
    tAxis = np.r_[0:Npulse]/fs

    # Create f(t) function 
    ft = ((f1 - f0) / ( 2 * (Npulse/fs)) ) * tAxis + f0

    # Analytic chirp response with amp 1
    pulse = np.exp(1j * 2 * np.pi * ft * tAxis)
    
    return pulse
    


* To validate that the function works display the pulse generated with Npulse = 200, f0=1000, f1 = 8000, fs = 48000. Remember the pulse is complex, so plot the real and imaginary part separately.

In [ ]:
%matplotlib inline
Npulse = 200
f0 = 1000
f1 = 8000
fs = 48000

# As provided
pulse = genChirpPulse(Npulse, f0, f1, fs)

# Create time axis
tAxis = np.r_[0:Npulse]/fs

width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.title("Real Plot")
plt.xlabel("N")
plt.ylabel("Amplitude")
plt.plot(tAxis[0:len(pulse)], np.real(pulse))


width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.title("Imaginary Plot")
plt.xlabel("N")
plt.ylabel("Amplitude")
plt.plot(tAxis, np.imag(pulse))

__Generate Pulse Trains__

Next, we will use the pulse generated by `genChirpPulse` and generate a pulse train.

* Write a new function `ptrain = genPulseTrain(pulse, Nrep, Nseg)`
The function accepts `pulse` = a pulse generated by `genChirpPulse`,  `Nrep` = number of pulse repetitions and `Nseg` = length of each pulse train segment (which is >= to the length of `pulse`).

The function returns `ptrain` which is a vector of length `Nrep` x `Nseg` (Hint: use `np.tile`)

In [8]:
def genPulseTrain(pulse, Nrep, Nseg):
    # Funtion generates a pulse train from a pulse. 
    #Inputs:
    #    pulse = the pulse generated by genChirpPulse
    #    Nrep  =  number of pulse repetitions
    #    Nseg  =  Length of pulse segment >= length(pulse)
    
    pulse = np.append(pulse, np.zeros(Nseg - len(pulse)))
    ptrain = np.tile(pulse, Nrep)
    
    return ptrain
  

### Part II, Task II: Echos in with Chirp pulse train

We now have components to generate pulses, generate a pulse train, play and record it. Lets see what we get!
We will start with very short pulses with a single carrier frequency. Rectangular pulses are difficult for the speaker
to produce as they exhibit discontinuities in the beginning and the end of the pulse. Therefore we will multiply the pulses
with a smooth window. Here, we will use a hanning window.

* Generate a f0=f1=8KHz, Npulse=96 pulse with fs=48000. Window the pulse with a hanning window. This will result in a pulse length of 2ms. You should be able to hear this tone.
* Plot the real and imaginary part of the pulse

In [ ]:
fs = 48000
f0 = 8000
f1 = 8000
Npulse = 96

# your code here:

# Create windowpulse with genChirp function - leverage np.hanning function
windowPulse = genChirpPulse(Npulse, f0, f1, fs)*np.hanning(Npulse)

# Create time axis
tAxis = np.r_[0:Npulse]

realWindowPulse = np.real(windowPulse)
imagWindowPulse = np.imag(windowPulse)

# Create figure for real function
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.xlabel("Time (ms)")
plt.ylabel("Magnitude")
plt.title("Windowed Chirp Pulse - Real")
plt.plot(tAxis/fs * 1000, realWindowPulse)

# Create figure for imaginary function
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.xlabel("Samples (N)")
plt.ylabel("Magnitude")
plt.title("Windowed Chirp Pulse - Imaginary")
plt.plot(tAxis, imagWindowPulse)

* Use the real part of the pulse to generate a pulse train of Nrep=15 pulses, Nseg=4096 samples

In [10]:
# your code here:
Nrep = 15
Nseg = 4096

# Use previous 'realWindowPulse' function
realPulseTrain = genPulseTrain(realWindowPulse, Nrep, Nseg)

* Play and record the pulse train. Scale the amplitude of the pulses to 1/2. Make sure your volume is set to maximum of 70% and look at the plot with the input pulse train and the received pulse train.

Use the pragma ``%matplotlib notebook`` for interactive plots, so you can zoom into the result.

In [11]:
# Scaling amplitude of the pulses to 1/2
rcv = xciever(realPulseTrain/2.0, fs) 

# I get error here: <ipython-input-18-74cf6cae67c5>:22: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
# me too, still works tho - Lewis

C:\Users\lewis_000\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:22: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [ ]:
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))

tAxis = np.r_[0:len(rcv)]/fs
# Plot generated pulse train
plt.plot(tAxis[0:len(realPulseTrain)], realPulseTrain/2.0,color='r')

# Plot received pulse train audio
plt.plot(tAxis[0:len(rcv)],rcv,color='k')

plt.xlabel("Time (s)")
plt.ylabel("Magnitude")
plt.title("Transmitted and Received Pulse Train")
plt.legend(["Transmitted Pulse Train (Speaker)", "Received Pulse Train (Mic)"],loc=1)
plt.minorticks_on()

* Extract a single pulse from the received pulse train. You can find the pulse index from the interactive plot. Extract at least 2 Npulse samples before the pulse and 20 Npulse samples after using `rcv_pulse = rcv[idx-2*Npulse:idx+Npulse*20]` 

* Plot the received pulse. Can you see any echoes?

You can disable interactivity by the pragma ``matplotlib inline``


In [ ]:
%matplotlib inline

# your code here:
NPulseSamples = 20
NPulseExtract = 2
# Estimate of pulse - couldn't get the interactive chart to work.

id_second = 0.62 # ENTER the time before a pulse in seconds <<--

idx = int(id_second*fs) # idx = 14000
# find index of start pulse - idx and Npulse params already set
rcv_pulse = rcv[idx-NPulseExtract*Npulse:idx+Npulse*NPulseSamples]

# make plot
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.xlabel("Time (ms)")
plt.ylabel("Magnitude")
plt.title("Single Pulse from Pulse Train")
timeAxis = np.r_[idx-NPulseExtract*Npulse:idx+Npulse*NPulseSamples]/fs *1000
plt.plot(timeAxis[0:len(rcv_pulse)], rcv_pulse)
plt.minorticks_on()

Some echo is visible, trailing the main pulse, with around the same length of the pulse.

#### Matched Filtering

The strong pulses we see are a result of direct feed-through from the transmitter to the receiver that do not scatter off targets. The echoes we see are a result of echoes from reflecting surfaces. The problem in our setup is that we don't know the exact delay between the transmitter and the receive hardware (in PyAudio). Instead, we will assume that the travel time for sound between the speaker and the microphone is negligible and much smaller than scatering targets. We can then detect when the pulses start based on the direct feedthrough signal. This assumption is very good as long as your speaker is close to the microphone!

We will detect both the feedthrough and echoes using matched filtering. 

* Write a function `Xrcv = crossCorr( rcv, pulse_a )` to calculate the cross correlation (matched filter) of the received signal with the analytic function of the pulse.  You can use `signal.fftconvolve`
* Take the absolute value of `Xrcv` to recover its envelope. Call the result `Xrcv_a`.

Make sure the plot is interactive with ``matplotlib notebook``

In [15]:
def crossCorr( rcv, pulse_a ):
    # Funtion generates cross-correlation between rcv and pulse_a
    # Inputs:
    #    rcv - received signal
    #    pulse_a - analytic pulse
    # Output:
    #    Xrcv - cross-correlation between rcv and pulse_a
    Xrcv = signal.fftconvolve(rcv, pulse_a[::-1])
    return Xrcv

In [ ]:
# your code here:
pulse_a = windowPulse
# Call cross correlation (matched filter)
Xrcv = crossCorr(rcv, pulse_a)
Xrcv_a = abs(Xrcv)

tAxis = np.r_[0:len(Xrcv_a)]/fs

width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.xlabel("Time (s)")
plt.ylabel("Magnitude")
plt.title("Matched-filtered Received Pulse Train")
plt.minorticks_on()
plt.plot(tAxis[0:len(Xrcv_a)], Xrcv_a)

* Again, extract a single pulse from the received pulse train using the same index. Extract at least 2 Npulse samples before the pulse and 20 Npulse samples after. Plot the received pulse. Can you see any echoes?

In [ ]:
NPulseSamples = 20
NPulseExtract = 2

id_second = 0.62 # ENTER the time before a pulse in seconds <<--

idx = int(id_second*fs)

Xrcv_a_pulse = Xrcv_a[idx-NPulseExtract*Npulse:idx+Npulse*NPulseSamples]

# plot
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.xlabel("Time (ms)")
plt.ylabel("Magnitude")
plt.title("Single Pulse from Natched-Filtered Pulse Train")
timeAxis = np.r_[idx-NPulseExtract*Npulse:idx+Npulse*NPulseSamples]/fs *1000
plt.plot(timeAxis, Xrcv_a_pulse)
plt.minorticks_on()

The signal is far cleaner. The previously observed echoes still seem to be present in the form of a side-peak to the right of the main peak. This side-lobe is almost 5 times smaller than the main peak and would vary dramatically depending on the computer hardware setup used and nature of the room environment.

#### Sonar System

In order to automate the system and visualize the results we need a few more components. To extract the pulses we need to know the position of the first feedthrough pulse. 


* Write a function `idx = findDelay(Xrcv_a, Nseg)` that takes the result of the matched filtering and finds the index of the first feedthrough pulse. Try testing on the actual signal to check whether the function is correct. There are multiple ways of doing it. `Nseg` is not necessarily required.

In [19]:
def findDelay(Xrcv, Nseg):
    # finds the first pulse
    # Inputs:  
    #         Xrcv - the received matched filtered signal
    #         Nseg - length of a segment
    # Output:
    #          idx - index of the beginning of the first pulse
    peaks, properties = signal.find_peaks(Xrcv,prominence = 0.6)
    return peaks[0]


In [ ]:
idx = findDelay(Xrcv_a,Nseg)

# Plot an 'x' to mark the first peak at 'idx' :
tAxis = np.r_[0:len(Xrcv_a)]/fs
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.xlabel("Time (s)")
plt.ylabel("Magnitude")
plt.title("Matched-filtered Received Pulse Train")
plt.minorticks_on()
plt.plot(tAxis, Xrcv_a)
plt.plot(idx/fs, Xrcv_a[idx],'rx')

We now can correct for delays and detect echoes. The only thing left now is to convert the time between echoes into actual distance.

If we send a pulse of sound, we will get reflection echoes of that pulse. Detecting the echos and their time-of-flight will reveal their distance from the source, based on the speed of sound in air. The speed of sound in air is given by the following equation:

$$ v_s = 331.5\sqrt{1+T/273.15}~\mathrm{m/s}~,$$ 

where T is the temperature in degree celcius. 

* Create a function `t = dist2time( dist, temperature )` that takes in the distance to the target in cm and converts it into the time in seconds between the transmitted pulse and its echo. Remember the arrival time include the time to the target and back and therefore the time should be doubled. 
For example, for temperature = 20 celcius and dist = 400 cm, the time it takes is 0.023 secs.

* Create a function `dist = time2dist( t, temperature )` that takes in the time to the target in seconds and converts it into the distance in cm between the transmitted pulse and its echo. Remember the arrival time include the time to the target and back and therefore the time should be halfed. 

In [23]:
def dist2time( dist, temperature=21):
    # Converts distance in cm to time in secs
    # Inputs:
    # dist        - distance to object in cm
    # temperature - in celcius
    # Output:
    # t           - time in seconds between transmitted pulse and echo
    v_s = 331.5*np.sqrt(1+temperature/273.15)
    d_metres = dist/100 # cm to m
    return 2*d_metres/v_s
 

def  time2dist(t,temperature=21):
    # Converts time in seconds to distance in cm
    # Inputs:
    # t        - time of echo
    # temperature - in celcius
    # Output:
    # dist          - distance in cm of the target
    v_s = 331.5*np.sqrt(1+temperature/273.15)
    return (t/2)*v_s *100 # cm
 

In [24]:
t = dist2time(400, 20)
print(t)

dist = time2dist(t, 20)
print(dist)

0.02329496742157036
400.0


## A sonar (almost)

* The following function will use your functions to generate pulses and display the matched filtering of each pulse as intensity of a horizontal line in an image. If nothing is moving, you will be able to see constant vertical lines representing echos. If something is moving, you will be able to track the object's distance.  



In [301]:
# some code for you

def sortOfASonar(Npulse, f0, f1,fs, Nrep, Nseg):
    pulse_a = genChirpPulse(Npulse, f0, f1, fs)
    hanWin = np.hanning(Npulse)
    hanWin = np.reshape(hanWin, (Npulse,1))
    pulse_a = np.multiply(pulse_a,hanWin)
    pulse = np.real(pulse_a)
    ptrain = genPulseTrain(pulse, Nrep, Nseg)
    rcv = xciever(ptrain/2.0 , fs) 
    Xrcv_a = abs( crossCorr(rcv, pulse_a) )
    Xrcv_a = np.reshape(Xrcv_a, (1,len(Xrcv_a)))
    
    idx = findDelay(Xrcv_a,Nseg) 
    img = np.zeros((Nrep,Nseg))
    img[0,:] = Xrcv_a[0,idx:idx+Nseg]
    
    # Look for peak in each pulse in the pulse train to avoid drift between xmit and receive
    for n in range(1,Nrep):
       idxx = findDelay(Xrcv_a[0,idx+int(Nseg/2):idx+int(Nseg/2)+Nseg],Nseg)
       idx = idx + idxx + int(Nseg/2)
       img[n,:]=Xrcv_a[0,idx:idx+Nseg]
        
    
    return img


Now, use the function above to:  

* Generate a pulse train of 100 pulses. Each (hamming windowed) pulse should be length of 72 samples (1.5ms) and constant frequency of 8KHz. The spacing between puslses should be 0.1 seconds (Nseg=4800). 
* Display the image

In [305]:
Npulse = 72
f0 = 8000
f1 = 8000
fs = 48000
Nrep = 100
Nseg = 4800

img = sortOfASonar(Npulse, f0, f1,fs, Nrep, Nseg)


C:\Users\lewis_000\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:22: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Display the result. Pay attention to the width of the echos. 

In [ ]:
%matplotlib notebook
%matplotlib notebook

# display up to 2.5m approximately 700 samples at 21 degrees C in 48000 sampling rate

vmax = 0.3 # threshold -- lower will be able to see smaller echos

plt.imshow(img[:,0:700]/max(img.ravel()),vmax=vmax, aspect=10,cmap='gray',interpolation='bilinear',extent=(0,time2dist(700/48000),Nrep*Nseg/fs,0))
plt.xlabel('cm')
plt.ylabel('sec')

Now, repeat the experiment with a chirp length of Nseg = 360 samples, and a frequency sweep from 6KHz to 12KHz.
Pay attention to the resolution of the lines. 

In [ ]:
Npulse = 72*5
f0 = 6000
f1 = 12000
fs = 48000
Nrep = 100
Nseg = 4800
img = sortOfASonar(Npulse, f0, f1,fs, Nrep, Nseg)

In [31]:
%matplotlib notebook

# display up to 2.5m approximately 700 samples at 21 degrees C in 48000 sampling rate

vmax = 0.3 # threshold -- lower will be able to see smaller echos

plt.imshow(img[:,0:700]/max(img.ravel()),vmax=0.2, aspect=10,cmap='gray',interpolation='bilinear',extent=(0,time2dist(700/48000),Nrep*Nseg/fs,0))


NameError: name 'img' is not defined

Feel free to repeat while moving a target-- can you see the echoes changing? Try playing with different parameters. 




## A Real (time) Sonar
You now have a working sonar! It would be much easier though to play with different parameters if we automate things, so we created some wrappers for real-time plotting in a separate notebook (lab1-RealTime-Sonar). 

* Copy-and-paste the 5 functions you created, including genPulseTrain(), genChirpPulse(), crossCorr(), findDelay(), and dist2time(), to the specified code cell in the real-time Sonar lab.


### You are now ready to proceed to the Real-Time Sonar Lab